# Use AutoAI and batch deployment to predict credit risk with Watson Machine Learning REST API

This notebook contains steps and code to demonstrate support of AutoAI experiments in Watson Machine Learning Service. It introduces commands for getting data, training experiments, persisting pipelines, publishing models, deploying models and scoring.

Some familiarity with cURL is helpful. This notebook uses cURL examples.


## Learning goals

The learning goals of this notebook are:

-  Working with Watson Machine Learning experiments to train AutoAI models.
-  Downloading computed models to local storage.
-  Batch deployment and scoring of trained model.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)  
2.	[Experiment definition](#experiment_definition)  
3.	[Experiment Run](#run)  
4.	[Historical runs](#runs)  
5.	[Deploy and Score](#deploy_and_score)  
6.	[Cleaning](#cleaning) 
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).
-  Create a <a href="https://console.bluemix.net/catalog/infrastructure/cloud-object-storage" target="_blank" rel="noopener no referrer">Cloud Object Storage (COS)</a> instance (a lite plan is offered and information about how to order storage can be found <a href="https://console.bluemix.net/docs/services/cloud-object-storage/basics/order-storage.html#order-storage" target="_blank" rel="noopener no referrer">here</a>). <br/>**Note: When using Watson Studio, you already have a COS instance associated with the project you are running the notebook in.**

You can find your COS credentials in COS instance dashboard under the **Service credentials** tab.
Go to the **Endpoint** tab in the COS instance's dashboard to get the endpoint information.

Authenticate the Watson Machine Learning service on IBM Cloud.

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.  

**NOTE:** You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.  

**Action:** Fill below information to get started with this notebook.


In [1]:
from json import loads
from IPython.display import JSON

In [ ]:
%env API_KEY=...
%env WML_ENDPOINT_URL=...
%env WML_INSTANCE_CRN="fill out only if you want to create a new space"
%env WML_INSTANCE_NAME=...

%env COS_CRN="fill out only if you want to create a new space"
%env COS_ENDPOINT=...
%env COS_BUCKET=...
%env COS_ACCESS_KEY_ID=...
%env COS_SECRET_ACCESS_KEY=...
%env COS_API_KEY=...

%env SPACE_ID="fill out only if you have space already created"

%env DATAPLATFORM_URL=https://api.dataplatform.cloud.ibm.com   
%env AUTH_ENDPOINT=https://iam.cloud.ibm.com/oidc/token

<a id="wml_token"></a>
### Getting WML authorization token for further cURL calls

<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-token" target="_blank" rel="noopener no referrer">Example of cURL call to get WML token</a>

In [3]:
%%bash --out token

curl -sk -X POST \
    --header "Content-Type: application/x-www-form-urlencoded" \
    --header "Accept: application/json" \
    --data-urlencode "grant_type=urn:ibm:params:oauth:grant-type:apikey" \
    --data-urlencode "apikey=$API_KEY" \
    "$AUTH_ENDPOINT" \
    | cut -d '"' -f 4

In [4]:
%env TOKEN=$token 

env: TOKEN=eyJraWQiOiIyMDIyMDMxNzA4MjMiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJJQk1pZC01NTAwMDA5MVZDIiwiaWQiOiJJQk1pZC01NTAwMDA5MVZDIiwicmVhbG1pZCI6IklCTWlkIiwianRpIjoiODAzZmEwMzctNWJkMC00YWZhLTljZTYtYTcwMDFjZGE1NDIxIiwiaWRlbnRpZmllciI6IjU1MDAwMDkxVkMiLCJnaXZlbl9uYW1lIjoiV01MIiwiZmFtaWx5X25hbWUiOiJXTUwtQmV0YSIsIm5hbWUiOiJXTUwgV01MLUJldGEiLCJlbWFpbCI6IldNTC1CZXRhQHBsLmlibS5jb20iLCJzdWIiOiJXTUwtQmV0YUBwbC5pYm0uY29tIiwiYXV0aG4iOnsic3ViIjoiV01MLUJldGFAcGwuaWJtLmNvbSIsImlhbV9pZCI6IklCTWlkLTU1MDAwMDkxVkMiLCJuYW1lIjoiV01MIFdNTC1CZXRhIiwiZ2l2ZW5fbmFtZSI6IldNTCIsImZhbWlseV9uYW1lIjoiV01MLUJldGEiLCJlbWFpbCI6IldNTC1CZXRhQHBsLmlibS5jb20ifSwiYWNjb3VudCI6eyJib3VuZGFyeSI6Imdsb2JhbCIsInZhbGlkIjp0cnVlLCJic3MiOiJlMGY3ZWMzYWMxYjI0ZWM5YWU3NzFlZmQ3NzI1MzhhMiIsImltc191c2VyX2lkIjoiNzE3Njk0MyIsImZyb3plbiI6dHJ1ZSwiaW1zIjoiMTY4NDc4MyJ9LCJpYXQiOjE2NDk3NjkzNDcsImV4cCI6MTY0OTc3Mjk0NywiaXNzIjoiaHR0cHM6Ly9pYW0uY2xvdWQuaWJtLmNvbS9pZGVudGl0eSIsImdyYW50X3R5cGUiOiJ1cm46aWJtOnBhcmFtczpvYXV0aDpncmFudC10eXBlOmFwaWtleSIsInNjb3Bl

<a id="space_creation"></a>
### Space creation
**Tip:** If you do not have `space` already created, please convert below three cells to `code` and run them.

First of all, you need to create a `space` that will be used in all of your further cURL calls. 
If you do not have `space` already created, below is the cURL call to create one.

<a href="https://cloud.ibm.com/apidocs/data-ai-common-core#spaces-create" 
target="_blank" rel="noopener no referrer">Space creation</a>

Space creation is asynchronous. This means that you need to check space creation status after creation call.
Make sure that your newly created space is `active`.

<a href="https://cloud.ibm.com/apidocs/data-ai-common-core#spaces-get" 
target="_blank" rel="noopener no referrer">Get space information</a>

<a id="experiment_definition"></a>
## 2. Experiment / optimizer configuration

Provide input information for AutoAI experiment / optimizer:
- `name` - experiment name
- `learning_type` - type of the problem
- `label` - target column name
- `scorer_for_ranking` - optimization metric
- `holdout_param` - percentage of training data to use as a holdout [0 - 1]
- `daub_include_only_estimators` - list of estimators to use

You can modify `parameters` section of the following cURL call to change AutoAI experiment settings.

<a href="https://cloud.ibm.com/apidocs/machine-learning#pipelines-create" 
target="_blank" rel="noopener no referrer">Define AutoAI experiment.</a>

In [8]:
%%bash --out pipeline_payload

PIPELINE_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "name": "Credit Risk Prediction - AutoAI", "description": "", "document": {"doc_type": "pipeline", "version": "2.0", "pipelines": [{"id": "autoai", "runtime_ref": "hybrid", "nodes": [{"id": "automl", "type": "execution_node", "parameters": {"stage_flag": true, "output_logs": true, "input_file_separator": ",", "optimization": {"learning_type": "binary", "label": "Risk", "max_num_daub_ensembles": 1, "daub_include_only_estimators": ["ExtraTreesClassifierEstimator", "GradientBoostingClassifierEstimator", "LGBMClassifierEstimator", "LogisticRegressionEstimator", "RandomForestClassifierEstimator", "XGBClassifierEstimator", "DecisionTreeClassifierEstimator"], "scorer_for_ranking": "roc_auc", "compute_pipeline_notebooks_flag": true, "run_cognito_flag": true, "holdout_param": 0.1}}, "runtime_ref": "autoai", "op": "kube"}]}], "runtimes": [{"id": "autoai", "name": "auto_ai.kb", "app_data": {"wml_data": {"hardware_spec": { "name": "L"}}}, "version": "3.0.2"}],"primary_pipeline": "autoai"}}'
echo $PIPELINE_PAYLOAD

In [9]:
JSON(loads(pipeline_payload))

<IPython.core.display.JSON object>

In [ ]:
%env PIPELINE_PAYLOAD=$pipeline_payload

In [11]:
%%bash --out pipeline_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$PIPELINE_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/pipelines?version=2020-08-01" \
    | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 5p

In [12]:
%env PIPELINE_ID=$pipeline_id

env: PIPELINE_ID=fa6d06e8-e1ee-4c17-a2bb-8482c4a5b3c8


<a id="experiment_details"></a>
### Get experiment details
To retrieve AutoAI experiment / optimizer configuration you can follow below cURL GET call.

<a href="https://cloud.ibm.com/apidocs/machine-learning#pipelines-get" 
target="_blank" rel="noopener no referrer">Get experiment / optimizer information</a>

In [13]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/pipelines/$PIPELINE_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "document": {
            "doc_type": "pipeline",
            "pipelines": [
                {
                    "id": "autoai",
                    "nodes": [
                        {
                            "id": "automl",
                            "op": "kube",
                            "parameters": {
                                "input_file_separator": ",",
                                "optimization": {
                                    "compute_pipeline_notebooks_flag": true,
                                    "daub_include_only_estimators": [
                                        "ExtraTreesClassifierEstimator",
                                        "GradientBoostingClassifierEstimator",
                                        "LGBMClassifierEstimator",
                                        "LogisticRegressionEstimator",
                                        "RandomForestClassifierEstimator",
                                 

<a id="training_connection"></a>
### Training data connection

Define connection information to COS bucket and training data CSV file. This example uses the German Credit Risk dataset. 

The dataset can be downloaded from [here](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/data/credit_risk/credit_risk_training_light.csv). You can also download it to local filesystem by running the cell below.

**Action**: Upload training data to COS bucket and enter location information in the next cURL examples.

In [14]:
%%bash

wget -q https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/data/credit_risk/credit_risk_training_light.csv \
     -O credit_risk_training_light.csv

<a id="cos_token"></a>
### Get COS token
Retrieve COS token for further authentication calls.

<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-token" 
target="_blank" rel="noopener no referrer">Retrieve COS authentication token</a>

In [15]:
%%bash --out cos_token

curl -s -X "POST" "$AUTH_ENDPOINT" \
     -H 'Accept: application/json' \
     -H 'Content-Type: application/x-www-form-urlencoded' \
     --data-urlencode "apikey=$COS_API_KEY" \
     --data-urlencode "response_type=cloud_iam" \
     --data-urlencode "grant_type=urn:ibm:params:oauth:grant-type:apikey" \
     | cut -d '"' -f 4

In [16]:
%env COS_TOKEN=$cos_token

env: COS_TOKEN=eyJraWQiOiIyMDIyMDMxNzA4MjMiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJpYW0tU2VydmljZUlkLTE1ODE3MTAyLTAxYWYtNDlhMC1iMTYxLWZjMGM2YTRhMjZlOCIsImlkIjoiaWFtLVNlcnZpY2VJZC0xNTgxNzEwMi0wMWFmLTQ5YTAtYjE2MS1mYzBjNmE0YTI2ZTgiLCJyZWFsbWlkIjoiaWFtIiwianRpIjoiMmUyYjhkZGUtZjMzOS00NGE0LWFiMDQtOWZiYmUyMDhkMzQ4IiwiaWRlbnRpZmllciI6IlNlcnZpY2VJZC0xNTgxNzEwMi0wMWFmLTQ5YTAtYjE2MS1mYzBjNmE0YTI2ZTgiLCJuYW1lIjoidGVzdC13bWwtc2FtcGxlcyIsInN1YiI6IlNlcnZpY2VJZC0xNTgxNzEwMi0wMWFmLTQ5YTAtYjE2MS1mYzBjNmE0YTI2ZTgiLCJzdWJfdHlwZSI6IlNlcnZpY2VJZCIsImF1dGhuIjp7InN1YiI6IlNlcnZpY2VJZC0xNTgxNzEwMi0wMWFmLTQ5YTAtYjE2MS1mYzBjNmE0YTI2ZTgiLCJpYW1faWQiOiJpYW0tU2VydmljZUlkLTE1ODE3MTAyLTAxYWYtNDlhMC1iMTYxLWZjMGM2YTRhMjZlOCIsInN1Yl90eXBlIjoiU2VydmljZUlkIiwibmFtZSI6InRlc3Qtd21sLXNhbXBsZXMifSwiYWNjb3VudCI6eyJib3VuZGFyeSI6Imdsb2JhbCIsInZhbGlkIjp0cnVlLCJic3MiOiJlMGY3ZWMzYWMxYjI0ZWM5YWU3NzFlZmQ3NzI1MzhhMiIsImZyb3plbiI6dHJ1ZX0sImlhdCI6MTY0OTc2OTQyMywiZXhwIjoxNjQ5NzczMDIzLCJpc3MiOiJodHRwczovL2lhbS5jbG91ZC5pYm0uY29tL2lkZW50aXR5Iiwi

<a id="cos_upload"></a>
### Upload file to COS
Upload your local dataset into your COS bucket

<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-put-object" 
target="_blank" rel="noopener no referrer">Upload file to COS</a>

In [17]:
%%bash

curl -sk -X PUT \
    --header "Authorization: Bearer $COS_TOKEN" \
    --header "Content-Type: application/octet-stream" \
    --data-binary "@credit_risk_training_light.csv" \
    "$COS_ENDPOINT/$COS_BUCKET/credit_risk_training_light.csv"

There should be an empty response when upload finished succesfully.

### Create connection to COS

Created connection will be used in training as a reference to given COS location.

In [20]:
%%bash --out connection_payload

CONNECTION_PAYLOAD='{"name": "REST COS connection", "datasource_type": "193a97c1-4475-4a19-b90c-295c4fdc6517", "properties": {"bucket": "'"$COS_BUCKET"'", "access_key": "'"$COS_ACCESS_KEY_ID"'", "secret_key": "'"$COS_SECRET_ACCESS_KEY"'", "iam_url": "'"$AUTH_ENDPOINT"'", "url": "'"$COS_ENDPOINT"'"}, "origin_country": "US"}'
echo $CONNECTION_PAYLOAD 

In [ ]:
JSON(loads(connection_payload))

In [ ]:
%env CONNECTION_PAYLOAD=$connection_payload

In [23]:
%%bash --out connection_id
CONNECTION_ID=$(curl -sk -X POST \
                    --header "Authorization: Bearer $TOKEN" \
                    --header "Content-Type: application/json" \
                    --header "Accept: application/json" \
                    --data "$CONNECTION_PAYLOAD" \
                    "$DATAPLATFORM_URL/v2/connections?space_id=$SPACE_ID")

CONNECTION_ID=${CONNECTION_ID#*asset_id\":\"}
CONNECTION_ID=${CONNECTION_ID%%\"*}
echo $CONNECTION_ID

In [24]:
%env CONNECTION_ID=$connection_id

env: CONNECTION_ID=fcda6643-799d-443f-bbe1-fabcd838b9bf


### Connections to external databases.

If you want to create experiment using data saved in the external database instead of COS,  
you should change the following properties in the `CONNECTION_PAYLOAD`:
 - `datasource_type`
 - `properties`  


Where `properties` key should contain credentials required for the connection creation which should follow the bellow pattern:
```
"properties": {
    "database": "***",
    "password": "***",
    "port": "***",
    "host": "***",
    "ssl": "***",
    "username": "***"
}
```
The `datasource_type` list can be obtained by using bellow request.

#### Exemplary payload for **DB2** database:

```
{
    "name": "REST DB2 connection",
    "datasource_type": "8c1a4480-1c29-4b33-9086-9cb799d7b157",
    "properties": {
        "database": "BLUDB",
        "username": "***"
        "password": "***",
        "port": 50000,
        "host": "dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net",
        "ssl": "true",
    },
    "origin_country": "US"
}

```

<a id="run"></a>
## 3. Experiment run

This section provides samples about how to trigger AutoAI experiment via cURL calls.

<a href="https://cloud.ibm.com/apidocs/machine-learning#trainings-create" 
target="_blank" rel="noopener no referrer">Schedule a training job for AutoAI experiment</a>

In [26]:
%%bash --out training_payload

TRAINING_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "training_data_references": [{"type": "connection_asset", "id": "credit_risk_training_light.csv", "connection": {"id": "'"$CONNECTION_ID"'"}, "location": {"bucket": "'"$COS_BUCKET"'", "file_name": "credit_risk_training_light.csv"}}], "results_reference": {"type": "connection_asset", "id": "autoai_results", "connection": {"id": "'"$CONNECTION_ID"'"}, "location": {"bucket": "'"$COS_BUCKET"'", "file_name": "."}}, "tags": [{"value": "autoai"}], "pipeline": {"id": "'"$PIPELINE_ID"'"}}'
echo $TRAINING_PAYLOAD

In [27]:
JSON(loads(training_payload))

<IPython.core.display.JSON object>

In [ ]:
%env TRAINING_PAYLOAD=$training_payload

In [29]:
%%bash --out training_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$TRAINING_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/trainings?version=2020-08-01" \
    | awk -F'"id":' '{print $2}' | cut -c2-37

In [30]:
%env TRAINING_ID=$training_id

env: TRAINING_ID=a88891fc-49d3-452f-996b-9578c2a73b63


<a id="training_details"></a>
### Get training details
Training is an asynchronous endpoint. In case you want to monitor training status and details,
you need to use a GET method and specify which training you want to monitor by usage of training ID.

<a href="https://cloud.ibm.com/apidocs/machine-learning#trainings-get" 
target="_blank" rel="noopener no referrer">Get information about training job</a>

### Get training status

In [32]:
%%bash

STATUS=$(curl -sk -X GET\
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/trainings/$TRAINING_ID?space_id=$SPACE_ID&version=2020-08-01")
    
STATUS=${STATUS#*state\":\"}
STATUS=${STATUS%%\"*}
echo $STATUS

completed


Please make sure that training is completed before you go to the next sections.
Monitor `state` of your training by running above cell couple of times.

<a id="runs"></a>
## 4. Historical runs

In this section you will see cURL examples describing how to get historical training runs information.

Output should be similar to the output from training creation but you should see more trainings entries.  
Listing trainings:

<a href="https://cloud.ibm.com/apidocs/machine-learning#trainings-list" 
target="_blank" rel="noopener no referrer">Get list of historical training jobs information</a>

In [ ]:
%%bash

HISTORICAL_TRAINING_LIMIT_TO_GET=2

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/trainings?space_id=$SPACE_ID&version=2020-08-01&limit=$HISTORICAL_TRAINING_LIMIT_TO_GET" \
    | python -m json.tool

<a id="training_cancel"></a>
### Cancel training run

**Tip:** If you want to cancel your training, please convert below cell to `code`, specify training ID and run.

<a href="https://cloud.ibm.com/apidocs/machine-learning#trainings-delete" 
target="_blank" rel="noopener no referrer">Canceling training</a>

---

<a id="deploy_and_score"></a>
## 5. Deploy and Score

In this section you will learn how to deploy and score pipeline model as webservice using WML instance.

Before deployment creation, you need store your model in WML repository.
Please see below cURL call example how to do it. Remember that you need to
specify where your chosen model is stored in COS.

<a id="model_store"></a>
### Store AutoAI model

Store information about your model to WML repository.

<a href="https://cloud.ibm.com/apidocs/machine-learning#models-create" 
target="_blank" rel="noopener no referrer">Model storing</a>

In [35]:
%%bash --out model_payload

MODEL_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "autoai_credit_risk_model","description": "This is description","type": "wml-hybrid_0.1", "software_spec": {"name": "hybrid_0.1"}, "content_location": { "type": "s3", "contents": [ { "content_format": "native", "file_name": "pipeline_model.json", "location": "'"$TRAINING_ID/assets/${TRAINING_ID}_P1_global_output/resources/wml_model/pipeline_model.json"'"}, { "content_format": "pipeline-node", "file_name": "P1_automl.zip", "location": "'"$TRAINING_ID/assets/${TRAINING_ID}_P1_global_output/resources/wml_model/P1_automl.zip"'", "pipeline_node_id": "automl"}],"connection": {"endpoint_url": "'"$COS_ENDPOINT"'", "access_key_id": "'"$COS_ACCESS_KEY_ID"'", "secret_access_key": "'"$COS_SECRET_ACCESS_KEY"'"}, "location": {"bucket": "'"$COS_BUCKET"'"}}}'
echo $MODEL_PAYLOAD

In [ ]:
JSON(loads(model_payload))

In [ ]:
%env MODEL_PAYLOAD=$model_payload

In [38]:
%%bash --out model_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$MODEL_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/models?version=2020-08-01" \
    | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p

In [39]:
%env MODEL_ID=$model_id

env: MODEL_ID=c1f8582c-3669-415d-b776-2a79e0618fb3


<a id="model_content_download"></a>
### Download model content

If you want to download your saved model, please make the following call.

<a href="https://cloud.ibm.com/apidocs/machine-learning#models-filtered-download" 
target="_blank" rel="noopener no referrer">Download model content</a>

In [37]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --output "model.tar.gz" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/download?space_id=$SPACE_ID&version=2020-08-01"

In [ ]:
!ls -l model.tar.gz

## <a id="deployment_creation"></a>
### Deployment creation

An AutoAI Batch deployment creation is presented below.

<a href="https://cloud.ibm.com/apidocs/machine-learning#deployments-create" 
target="_blank" rel="noopener no referrer">Create deployment</a>

In [41]:
%%bash --out deployment_payload

DFEPLOYMENT_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "AutoAI deployment","description": "This is description","batch": {}, "hybrid_pipeline_hardware_specs": [{"node_runtime_id": "auto_ai.kb", "hardware_spec": {"name": "M"}}],"asset": {"id": "'"$MODEL_ID"'"}}'
echo $DFEPLOYMENT_PAYLOAD

In [43]:
JSON(loads(deployment_payload))

<IPython.core.display.JSON object>

In [ ]:
%env DFEPLOYMENT_PAYLOAD=$deployment_payload

In [44]:
%%bash --out deployment_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$DFEPLOYMENT_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/deployments?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p

In [45]:
%env DEPLOYMENT_ID=$deployment_id

env: DEPLOYMENT_ID=23a726a1-9853-49d8-834e-db301d813267


<a id="deployment_details"></a>
### Get deployment details
As deployment API is asynchronous, please make sure your deployment is in `ready` state before going to the next points.

<a href="https://cloud.ibm.com/apidocs/machine-learning#deployments-get" 
target="_blank" rel="noopener no referrer">Get deployment details</a>

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/deployments/$DEPLOYMENT_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

<a id="batch_score"></a>
### Score your Batch deployment
Scoring for Batch deployment is done by creating `jobs`. User can specify job payload as a json or as data connection to eg. COS.

<a href="https://cloud.ibm.com/apidocs/machine-learning#deployment-jobs-create" 
target="_blank" rel="noopener no referrer">Create deployment job</a>

In [47]:
%%bash --out job_payload

JOB_PAYLOAD='{"name": "AutoAI job", "space_id": "'"$SPACE_ID"'","deployment": {"id": "'"$DEPLOYMENT_ID"'"}, "hybrid_pipeline_hardware_specs": [{"node_runtime_id": "auto_ai.kb", "hardware_spec": {"name": "M"}}], "scoring": {"input_data": [{"fields": ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings", "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration", "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents", "Telephone", "ForeignWorker"], "values": [["less_0", 6, "all_credits_paid_back", "car_used", 250, "less_100", "1_to_4", 2, "male", "none", 2, "savings_insurance", 28, "stores", "rent", 1, "skilled", 1, "none", "yes"]]}]}}'
echo $JOB_PAYLOAD 

In [49]:
JSON(loads(job_payload))

<IPython.core.display.JSON object>

In [ ]:
%env JOB_PAYLOAD=$job_payload

In [50]:
%%bash --out job_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$JOB_PAYLOAD" \
    "$WML_ENDPOINT_URL/ml/v4/deployment_jobs?version=2020-08-01" \
    | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p

In [51]:
%env JOB_ID=$job_id

env: JOB_ID=45ccfd00-2518-4a84-9aeb-0de5f9a45680


<a id="job_list"></a>
### Listing all Batch jobs

<a href="https://cloud.ibm.com/apidocs/machine-learning#deployment-jobs-list" 
target="_blank" rel="noopener no referrer">List jobs</a>

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/deployment_jobs?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

<a id="job_get"></a>
### Get particular job details

<a href="https://cloud.ibm.com/apidocs/machine-learning#deployment-jobs-get" 
target="_blank" rel="noopener no referrer">Get job details</a>

In [55]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/deployment_jobs/$JOB_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "deployment": {
            "id": "23a726a1-9853-49d8-834e-db301d813267"
        },
        "platform_job": {
            "job_id": "74803edb-0727-464b-9931-4343b02efb30",
            "run_id": "7af48a72-761e-4cc8-9f39-7bf8be4ac30a"
        },
        "scoring": {
            "input_data": [
                {
                    "fields": [
                        "CheckingStatus",
                        "LoanDuration",
                        "CreditHistory",
                        "LoanPurpose",
                        "LoanAmount",
                        "ExistingSavings",
                        "EmploymentDuration",
                        "InstallmentPercent",
                        "Sex",
                        "OthersOnLoan",
                        "CurrentResidenceDuration",
                        "OwnsProperty",
                        "Age",
                        "InstallmentPlans",
                        "Housing",
                       

<a id="job_cancel"></a>
### Cancel job

**Tip:** You can cancel running job by calling delete method.
Just convert below cell to `code` and run it.

<a href="https://cloud.ibm.com/apidocs/machine-learning#deployment-jobs-delete" 
target="_blank" rel="noopener no referrer">Cancel job</a>

<a id="deployments_list"></a>
### Listing all deployments

<a href="https://cloud.ibm.com/apidocs/machine-learning#deployments-list" 
target="_blank" rel="noopener no referrer">List deployments details</a>

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/deployments?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

<a id="cleaning"></a>
## 6. Cleaning section

Below section is useful when you want to clean all of your previous work within this notebook.
Just convert below cells into the `code` and run them.

<a id="training_delete"></a>
### Delete training run
**Tip:** You can completely delete a training run with its metadata.

<a href="https://cloud.ibm.com/apidocs/machine-learning#trainings-delete" 
target="_blank" rel="noopener no referrer">Deleting training</a>

<a id="job_delete"></a>
### Delete job

**Tip:** If you want remove job completely (with metadata), just specify `hard_delete` to True.

<a href="https://cloud.ibm.com/apidocs/machine-learning#deployment-jobs-delete" 
target="_blank" rel="noopener no referrer">Delete job</a>

<a id="deployment_delete"></a>
### Deleting deployment
**Tip:** You can delete existing deployment by calling DELETE method.

<a id="model_delete"></a>
### Delete model from repository
**Tip:** If you want to completely remove your stored model and model metadata, just use a DELETE method.

<a href="https://cloud.ibm.com/apidocs/machine-learning#models-delete" 
target="_blank" rel="noopener no referrer">Delete model from repository</a>

<a id="summary"></a>
## 7. Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to use `cURL` calls to store, deploy and score a AutoAI model in WML. 


### Authors

**Amadeusz Masny**, Python Software Developer in Watson Machine Learning at IBM  
**Jan Sołtysik**, Intern in Watson Machine Learning at IBM

Copyright © 2020, 2021, 2022 IBM. This notebook and its source code are released under the terms of the MIT License.